In [ ]:
from dataread import *
from DATA_INDICATORS import *
from sklearn.preprocessing import MinMaxScaler
import joblib
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from keras.layers import Dense,LSTM
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os

In [ ]:
def add_target(data):
    target = []
    target.append(None)
    for i in range(1, len(data)):
        if data.iloc[i]['close'] >= data.iloc[i-1]['close']:
            target.append(1)
        else:
            target.append(0)
    data['target'] = target
    return data

In [ ]:
def make_dataset(data, label, window_size=5):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [ ]:
ticker = download_data().Usdt_symbol()
# ticker = ticker[142:]
for i in ticker:
    try:
        print("ticker :",str(i))
        data = download_data(code_name = i,frame='1d').get_data()
        data = add_ma(data)
        data = add_rsi(data)
        data = add_ema(data)
        data = add_stochastic(data)
        data = add_disparity(data)
        data = data.dropna()
        # scale
        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(data)

        df_scaled = pd.DataFrame(df_scaled)
        df_scaled.columns = data.columns

        # slicing data
        train_feature = pd.DataFrame(df_scaled.drop(['close'],axis=1))
        train_label = pd.DataFrame(df_scaled['close'])
        train_feature, train_label = make_dataset(train_feature, train_label, window_size = 20)
        x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2,random_state=0)

        # create model
        try:
            model = Sequential()
            model.add(LSTM(16, 
                        input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                        activation='softsign', 
                        return_sequences=True)
                    )
            model.add(Dense(8))
            model.add(Dense(4))
            model.add(Dense(1))
            try:
                model_path = 'model_path'
                model.compile(loss='mean_squared_error', optimizer='adam')
                early_stop = EarlyStopping(monitor='val_loss', patience=5)
                filename = os.path.join(model_path, 'tmp_checkpoint.h5')
                checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

                history = model.fit(x_train, y_train, 
                                    epochs=200, 
                                    batch_size=16,
                                    validation_data=(x_valid, y_valid), 
                                    callbacks=[early_stop, checkpoint])
            except:
                print("model compile error")
        except:
            print("model create error")
    except:
        print("data read error")
    # 모델 저장
    try:
        joblib.dump(model, './model_'+ str(i).replace("/","_") +'.pkl')
    except:
        print("model save error")
    # model = joblib.load('saved_model.pkl')